In [6]:
library(tidyverse)

In [7]:
# Make simple table for analysis
Total.table <- read.csv("01.DESeq_results/total_rh_og_gene.csv")
Total.simple.table <- Total.table %>%
  dplyr::select(Gene, starts_with("log2"), starts_with("padj"))

#Import GO database
GOdata<-read.csv('Reference/Cleaned_GO_4.csv', sep=';', header=FALSE)
GOdata$Gene<-as.character(GOdata$V1)
GOdata$V1<-NULL
GOdata$geneabb<-as.character(GOdata$V2)
GOdata$V2<-NULL
GOdata$genename<-as.character(GOdata$V3)
GOdata$V3<-NULL
GOdata$GO<-as.character(GOdata$V4)
GOdata$V4<-NULL

Merged.table <- Total.simple.table %>%
  left_join(GOdata, by = "Gene") %>%
  mutate(
    geneabb = ifelse(is.na(geneabb), Gene, geneabb),   
    genename = ifelse(is.na(genename), "", genename),   
    GO = ifelse(is.na(GO), "", GO)                    
  )

Merged.table <- Merged.table %>%
  dplyr::select(
    Gene, geneabb, genename, 
    log2FoldChange_RHAM_30min, padj_RHAM_30min,
    log2FoldChange_RHAM_90min, padj_RHAM_90min, 
    log2FoldChange_OGs_30min, padj_OGs_30min, 
    log2FoldChange_OGs_90min, padj_OGs_90min, 
    GO
  )

colnames(Merged.table) <- c("Gene", "Geneabb", "Genename", 
                            "Log2FC_RG_I_30", "Padj_RG_I_30", 
                            "Log2FC_RG_I_90","Padj_RG_I_90",
                            "Log2FC_OGs_30", "Padj_OGs_30",
                            "Log2FC_OGs_90", "Padj_OGs_90",
                            "GO")

colnames(Merged.table)

[1] "Gene"           "Geneabb"        "Genename"       "Log2FC_RG_I_30"
 [5] "Padj_RG_I_30"   "Log2FC_RG_I_90" "Padj_RG_I_90"   "Log2FC_OGs_30" 
 [9] "Padj_OGs_30"    "Log2FC_OGs_90"  "Padj_OGs_90"    "GO"

In [8]:
#set the threshold
Th.lfc2 <- 1
Th.padj <- 0.05

# Filter the table
rh30up <- Merged.table %>%
  dplyr::filter(Log2FC_RG_I_30 > Th.lfc2, Padj_RG_I_30 < Th.padj)
rh90up <- Merged.table %>%
  dplyr::filter(Log2FC_RG_I_90 > Th.lfc2, Padj_RG_I_90 < Th.padj)
og30up <- Merged.table %>%
  dplyr::filter(Log2FC_OGs_30 > Th.lfc2, Padj_OGs_30 < Th.padj)
og90up <- Merged.table %>%
  dplyr::filter(Log2FC_OGs_90 > Th.lfc2, Padj_OGs_90 < Th.padj)

# rg overlap gene
rh_up <- intersect(rh30up$Gene, rh90up$Gene)
length(rh_up)

rh_up_table <- Merged.table %>%
  dplyr::filter( Gene %in% rh_up)

# og overlap gene
og_up <- intersect(og30up$Gene, og90up$Gene)
length(og_up)

og_up_table <- Merged.table %>%
  dplyr::filter( Gene %in% og_up)

# both overlap gene
both_up <- intersect(rh_up, og_up)
length(both_up)

both_up_table <- Merged.table %>%
  dplyr::filter( Gene %in% both_up)

# Export tables
write_csv(rh_up_table, "07.Table/RG-I_both_up.csv")
print("01.DESeq_results/total_og_gene.csv")
write_csv(og_up_table, "07.Table/OGs_both_up.csv")
print("01.DESeq_results/total_rh_gene.csv")
write_csv(both_up_table, "07.Table/RG-I_OGs_both_up.csv")
print("01.DESeq_results/total_rh_og_gene.csv")


[1] 488

[1] 939

[1] 303

[1] "01.DESeq_results/total_og_gene.csv"
[1] "01.DESeq_results/total_rh_gene.csv"
[1] "01.DESeq_results/total_rh_og_gene.csv"
